In [ ]:
from tia.bbg import LocalTerminal
import tia.bbg.datamgr as dm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import textwrap
from pandas import ExcelWriter
import openpyxl
from openpyxl.utils import get_column_letter

mgr = dm.BbgDataManager()

In [ ]:
import bql
bq = bql.Service()

universe = bq.univ.members(type='AIM', accounts=[XXXXXX]

data_item = bq.data.ts_position()

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()


In [ ]:
data.head()

In [ ]:
data = data.groupby('ID').sum()

In [ ]:
ids = data.index.to_list()

In [ ]:
universe = ids

data_item1 = bq.data.MIFID_SUB_ASSET_CLASS()

request1 = bql.Request(universe, data_item1)


response1 = bq.execute(request1)

data1 = response1[0].df()


In [ ]:
mask = data1['MIFID_SUB_ASSET_CLASS()'].str.contains('CRPB', case=False, na=False)

In [ ]:
data1 = data1[mask]

In [ ]:
ids = data1.index.to_list()

In [ ]:
resp = LocalTerminal.get_reference_data(ids,["SECURITY_NAME","MARKET_SECTOR_DES","CAST_PARENT_NAME","CRNCY","IS_EMERGING_MARKET"],ignore_field_error=1)
tickers = resp.as_frame()

In [ ]:
issuers = ['BAC']

In [ ]:
tickers['issuers']=tickers['SECURITY_NAME'].str.split().str[0]

In [ ]:
tickers = tickers[tickers['issuers'].isin(issuers)]

In [ ]:
data1 = data1.merge(data, how='left', left_index=True, right_index=True)


In [ ]:
tickers = tickers.merge(data1,how='left', left_index=True, right_index=True)



In [ ]:
tickers.dropna(subset=['CAST_PARENT_NAME'], inplace=True)


In [ ]:
tickers['issuer + currency'] = tickers['CAST_PARENT_NAME'] + tickers['CRNCY']
tickers.drop_duplicates(subset=['issuer + currency'], inplace = True)

In [ ]:
tickers = tickers[tickers.IS_EMERGING_MARKET == 'N']
tickers

In [ ]:
# sample_tickers = tickers[tickers.CRNCY == 'EUR']

In [ ]:
# sample_tickers

In [ ]:
unique_tickers = tickers.index.to_list()
unique_tickers

In [ ]:
bonds = []
for ticker in unique_tickers:
    universe = bq.univ.filter(bq.univ.bonds(ticker),bq.func.and_(bq.data.crncy() == tickers.loc[ticker]['CRNCY'] ,bq.data.AMT_OUTSTANDING() > 150000000))
    
    data_item = bq.data.name()
    
    request = bql.Request(universe, data_item)
    response = bq.execute(request)

    fetch = response[0].df()
    bonds.append(fetch)

In [ ]:
final_dfs = []

for df in bonds:
    ids = df.index.tolist()
    resp = LocalTerminal.get_reference_data(ids,["ID_ISIN", "SECURITY_NAME", "CPN", "MATURITY", "AMT_OUTSTANDING", "YAS_MOD_DUR",
                                                "CNVX_MID", "YLD_CNV_MID", "YLD_CUR_MID", "YAS_ISPREAD_TO_GOVT", "YAS_OAS_SPRD", "LAST_PRICE", "CHG_PCT_YTD","YLD_CHG_YTD_MID_NET",
                                                 "RTG_SP", "RTG_MOODY", "RTG_FITCH", "NXT_CALL_DT", "NXT_CALL_PX", "CRNCY","NORMALIZED_PAYMENT_RANK"],ignore_field_error=1)
    resp_df = resp.as_frame()
    
    df = df.join(resp_df)
    df = df.sort_values(by='MATURITY', ascending=True)
    
    final_dfs.append(df)

In [ ]:
final_dfs_merged = []
for df in final_dfs:
    df = df.merge(data, how='left',left_index = True, right_index = True)
    move_col = df.pop('TS_POSITION()')
    df.insert(4,'TS_POSITION()',move_col)
    # df['ROOM_TO_10%'] = df["AMT_OUTSTANDING"]
    # move_col1 = df.pop('ROOM_TO_10%')
    # df.insert(8,'ROOM_TO_10',move_col)

    
    
    final_dfs_merged.append(df)

In [ ]:
final_dfs_merged

In [ ]:
writer = pd.ExcelWriter('usd_exposure.xlsx', engine='openpyxl')
for df in final_dfs_merged:
    name = df['NAME()'][0].split(" ")[0] + df['CRNCY']
    df.to_excel(writer, sheet_name=name, index=False)

    worksheet = writer.sheets[name]

    for col in df.columns:
        # Find the maximum length of the data in each column
        max_length = max(df[col].astype(str).apply(len).max(), len(col))
        # Adjust the column width; add a small buffer to ensure fit
        column_letter = get_column_letter(df.columns.get_loc(col) + 1)
        worksheet.column_dimensions[column_letter].width = max_length + 2

writer.save()